In [ ]:
#imports
import openai
import os
client = openai.OpenAI()
openai.api_key = os.getenv("OPENAI_API_KEY")


Taking into account token limits, the plan is to have the Cx paste documents into the chat, then the bot will summarize them and highlight differences.

In [ ]:
#user chat loop
messages=[]
completion = client.chat.completions.create(
  model="gpt-3.5-turbo",
  messages=[
    {"role": "system", "content": "You are a legal assistant, designed to simplify and highlight differences between legal documents."},
    {"role": "user", "content": "."}
  ]
)

resp = ''
while True:
    resp = input()
    if resp.lower() == 'stop' or resp.lower() == 'exit': break
    
    messages.append({"role": "user", "content": resp})
    print(completion.choices[0].message)
    
with open('outputs.txt', 'a') as out:
    print(list(messages.values))
    #out.writelines(list(messages.values)
    #out.write('\n')
    

In [ ]:
#LEDGAR dataset cleaning
import pandas as pd
import json
#LEDGAR dataset load/parse
lines = []
with open(r'LEDGAR_2016-2019_clean.jsonl', encoding='utf8') as line:
    lines = line.read().splitlines()

line_dicts = [json.loads(line) for line in lines]
df_final = pd.DataFrame(line_dicts)

print(df_final)

In [ ]:
import openai
import os 
from random import randint
import pandas as pd

UPPER_LIMIT = 1 #NUMBER OF LOOP RUNS
GPT_MODEL = "gpt-3.5-turbo" #GPT model to use, change to "gpt-4-turbo" for better performance
LOGGING = False #Write to document 

client = openai.OpenAI()
openai.api_key = os.getenv("OPENAI_API_KEY")

for i in range (0,UPPER_LIMIT):
    lst = os.listdir("CUAD_v1\\label_group_xlsx") #Retrieve random category from CUAD dataset
    rand_category = lst[randint(0,len(lst)-1)]
    print(rand_category)
    xlsx = pd.read_excel('CUAD_v1\\label_group_xlsx\\' + rand_category) #import excel to dataframe
    randint1 = randint(0, len(xlsx['Filename']) - 1)
    randint2 = randint(0, len(xlsx['Filename']) - 1)
    cont1 = xlsx['Filename'][randint1] #retrieve random contract filenames in txt
    cont2 = xlsx['Filename'][randint2]
    cont1 = cont1[:-3].replace("&", "_") + "txt"
    cont2 = cont2[:-3].replace("&", "_") + "txt"
    print(cont1[:10], cont2[:10])
    cont1TXT = ''
    cont2TXT = ''
    with open('CUAD_v1\\full_contract_txt\\' + cont1, "r", encoding='utf8') as out: cont1TXT = out.read() #contract text out
    with open('CUAD_v1\\full_contract_txt\\' + cont2, "r", encoding='utf8') as out: cont2TXT = out.read()   
    
    #Create GPT instance and prompt with "Name the clauses that differ between these two documents: "
    messages=[]
    completion = client.chat.completions.create(
        model=GPT_MODEL,
        messages=[
            {"role": "system", "content": "You are a legal assistant, designed to simplify and highlight differences between legal documents."},
            {"role": "user", "content": "."}
        ]
    )
    #Contract prompt
    messages.append({"role": "user", "content": ("Name the clauses that differ between these two documents: " + cont1TXT
    + "\n" + cont2TXT)})
    #print response
    print(completion.choices[0].message)
    if Logging:
        with open('outputs.txt', 'a') as out:
            out.writelines(list(messages.values))
            out.write('\n')
    print(list(messages))


In [ ]:
from docx.enum.text import WD_COLOR_INDEX
import docx
from re import finditer

In [ ]:
doc1 = ''
doc2 = ''
comparisonDocument1 = docx.Document()
comparisonDocument2 = docx.Document()
def output_parse(gptOut):
    []
for paragraph in document.split('\n'):
    if clauseNumber in paragraph:
        comparisonDocument1.add_paragraph(paragraph)